Detección de barras en galaxias - Modelos Alternativos

Proyecto integrador MNA

Integrantes

Jonathan Jesús Marmolejo Hernández - A01795195

Isaid Posadas Oropeza - A01795015

Luis Daniel Ortega Muñoz - A01795197


# Modelos de ensamble heterogéneos Blending y Stacking con YOLOv8s, YOLOv8m, YOLOv8l y YOLOv8x
      

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Instalar YOLOv8

In [2]:
!pip install ultralytics > /dev/null 2>&1

In [3]:
from ultralytics import YOLO

# Verificar versión
import ultralytics
print(f"Ultralytics versión: {ultralytics.__version__}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics versión: 8.3.147


In [4]:
from ultralytics import settings
settings.reset()  # Reinicia configuraciones de cache si hay conflictos

# Ensambles Blending y Stacking

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import pandas as pd
from ultralytics import YOLO
from tqdm import tqdm

# Rutas
model_paths = {
    'v8s': '/content/drive/MyDrive/MNA_ProyectoIntegrador/modelos_yolo/best_v8s_LogDiff.pt',
    'v8m': '/content/drive/MyDrive/MNA_ProyectoIntegrador/modelos_yolo/best_v8m_LogDiff.pt',
    'v8l': '/content/drive/MyDrive/MNA_ProyectoIntegrador/modelos_yolo/best_v8l_LogDiff.pt',
    'v8x': '/content/drive/MyDrive/MNA_ProyectoIntegrador/modelos_yolo/best_v8x_LogDiff.pt'
}

image_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/galaxy_dataset_2/images/train'
gt_label_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/galaxy_dataset_2/labels/train'
output_csv = '/content/drive/MyDrive/MNA_ProyectoIntegrador/stacking_blending_input_sample.csv'

# Parámetro de control
N = 1000  # Número de ejemplos por clase

# Verificar si el archivo ya existe
if os.path.exists(output_csv):
    print(f"Archivo ya existe: {output_csv}")
    df = pd.read_csv(output_csv)
    print("CSV cargado con éxito.")
else:
    # Clasificar imágenes por clase según etiquetas
    with_bar = []
    without_bar = []

    for img_name in os.listdir(image_dir):
        if not img_name.lower().endswith(('.jpg', '.png')):
            continue
        gt_file = os.path.join(gt_label_dir, os.path.splitext(img_name)[0] + '.txt')
        if os.path.exists(gt_file):
            with open(gt_file, 'r') as f:
                lines = f.readlines()
            (with_bar if len(lines) > 0 else without_bar).append(img_name)
        else:
            without_bar.append(img_name)

    # Balancear
    selected_with = with_bar[:N]
    selected_without = without_bar[:N]
    selected_filenames = selected_with + selected_without

    print(f"Total imágenes seleccionadas: {len(selected_filenames)} (con y sin barra balanceadas)")

    # Cargar modelos
    models = {name: YOLO(path) for name, path in model_paths.items()}

    # Procesar imágenes
    data = []

    for img_name in tqdm(selected_filenames, desc="Procesando muestra balanceada"):
        row = {'image': img_name}
        img_path = os.path.join(image_dir, img_name)

        img = cv2.imread(img_path)
        if img is None:
            print(f"No se pudo leer la imagen: {img_path}")
            continue

        for name, model in models.items():
            results = model(img)
            boxes = results[0].boxes

            confs = []
            if boxes is not None:
                for i in range(len(boxes)):
                    cls = int(boxes.cls[i].cpu().item())
                    conf = float(boxes.conf[i].cpu().item())
                    if cls == 0:
                        confs.append(conf)

            row[f'pred_{name}'] = max(confs) if confs else 0.0

        row['true_label'] = 1 if img_name in selected_with else 0
        data.append(row)

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    print(f"CSV generado con muestra balanceada: {output_csv}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total imágenes seleccionadas: 2000 (con y sin barra balanceadas)


Procesando muestra balanceada:   0%|          | 0/2000 [00:00<?, ?it/s]


0: 640x640 1 barra, 8.7ms
Speed: 18.0ms preprocess, 8.7ms inference, 401.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 barra, 10.0ms
Speed: 2.1ms preprocess, 10.0ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 barra, 11.6ms
Speed: 2.0ms preprocess, 11.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 barras, 13.7ms
Speed: 2.1ms preprocess, 13.7ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)


Procesando muestra balanceada:   0%|          | 1/2000 [00:09<5:28:18,  9.85s/it]


0: 640x640 1 barra, 10.0ms
Speed: 2.3ms preprocess, 10.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 barra, 9.7ms
Speed: 1.9ms preprocess, 9.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 barra, 11.7ms
Speed: 1.8ms preprocess, 11.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 barra, 11.8ms
Speed: 2.1ms preprocess, 11.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


...


0: 640x640 1 barra, 8.4ms
Speed: 2.2ms preprocess, 8.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 barra, 10.0ms
Speed: 1.9ms preprocess, 10.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 barra, 11.5ms
Speed: 1.9ms preprocess, 11.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 barras, 11.8ms
Speed: 1.7ms preprocess, 11.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


Procesando muestra balanceada: 100%|█████████▉| 1999/2000 [17:13<00:00,  2.29it/s]


0: 640x640 1 barra, 11.2ms
Speed: 2.9ms preprocess, 11.2ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 barra, 14.4ms
Speed: 2.9ms preprocess, 14.4ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 barra, 12.9ms
Speed: 2.8ms preprocess, 12.9ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 barra, 12.7ms
Speed: 2.0ms preprocess, 12.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


Procesando muestra balanceada: 100%|██████████| 2000/2000 [17:14<00:00,  1.93it/s]

CSV generado con muestra balanceada: /content/drive/MyDrive/MNA_ProyectoIntegrador/stacking_blending_input_sample.csv


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar CSV generado
df = pd.read_csv('/content/drive/MyDrive/MNA_ProyectoIntegrador/stacking_blending_input_sample.csv')

# Entradas: predicciones de los modelos base
X = df[['pred_v8s', 'pred_v8m', 'pred_v8l', 'pred_v8x']]
y = df['true_label']

# División entrenamiento/validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


## Blending (Regresión logística sobre predicciones)

In [8]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Importar librerías necesarias
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib
import os

# 3. Entrenar el modelo
blender = LogisticRegression()
blender.fit(X_train, y_train)

# 4. Predecir
blend_pred = blender.predict(X_val)

# 5. Evaluar
print("=== Blending ===")
print(classification_report(y_val, blend_pred, digits=4))

# 6. Guardar el modelo en Drive
output_path = '/content/drive/MyDrive/MNA_ProyectoIntegrador/modelos_ensamble/blending_model.pkl'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
joblib.dump(blender, output_path)

print(f'Modelo guardado en: {output_path}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== Blending ===
              precision    recall  f1-score   support

           0     0.7143    0.6500    0.6806       200
           1     0.6789    0.7400    0.7081       200

    accuracy                         0.6950       400
   macro avg     0.6966    0.6950    0.6944       400
weighted avg     0.6966    0.6950    0.6944       400

Modelo guardado en: /content/drive/MyDrive/MNA_ProyectoIntegrador/modelos_ensamble/blending_model.pkl


## Stacking (Ensamble con meta-modelo)

In [9]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Importar librerías necesarias
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib
import os

# 3. Definir estimadores base y meta-modelo
estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(probability=True))
]

# 4. Crear modelo de stacking
stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# 5. Entrenar modelo
stack.fit(X_train, y_train)

# 6. Predecir
stack_pred = stack.predict(X_val)

# 7. Evaluar
print("=== Stacking ===")
print(classification_report(y_val, stack_pred, digits=4))

# 8. Guardar modelo en Google Drive
output_path = '/content/drive/MyDrive/MNA_ProyectoIntegrador/modelos_ensamble/stacking_model.pkl'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
joblib.dump(stack, output_path)

print(f'Modelo guardado en: {output_path}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== Stacking ===
              precision    recall  f1-score   support

           0     0.7386    0.6500    0.6915       200
           1     0.6875    0.7700    0.7264       200

    accuracy                         0.7100       400
   macro avg     0.7131    0.7100    0.7090       400
weighted avg     0.7131    0.7100    0.7090       400

Modelo guardado en: /content/drive/MyDrive/MNA_ProyectoIntegrador/modelos_ensamble/stacking_model.pkl
